<a href="https://colab.research.google.com/github/SarthakAgase/AI-Speech-Emotion-Detection/blob/main/ML_Text_To_Emotion_with_all_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Emotion Detection Models




## Data Preprocessing

### Importing the libraries

In [ ]:
import pandas as pd
import numpy as np
import re, nltk

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report


nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

stop_words = set(stopwords.words('english'))
stop_words.remove("no")
stop_words.remove("not")
stop_words.remove("nor")
stop_words = [x.lower() for x in stop_words]

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [ ]:
!pip install lightgbm
!pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 8.9 MB/s eta 0:00:00


### Pre-Defined Dictionary Words

In [ ]:
contractions = {
        "ai\snot": "am not",
        "wo\snot": "will not",
        "would\snot": "would not",
        "should\snot": "should not",
        "isn\snot": "is not",
        "aren\snot": "are not",
        "wasn\snot": "was not",
        "weren\snot": "were not",
        "haven\snot": "have not",
        "hasn\snot": "has not",
        "hadn\snot": "had not",
        "don\snot": "do not",
        "doesn\snot": "does not",
        "didn\snot": "did not",
        "can\snot": "can not",
        "cannot": "can not",
        "couldn\snot": "could not",
        "shouldn\snot": "should not",
        "mightn\snot": "might not",
        "mustn\snot": "must not",
        "shan\snot": "shall not",
        "won\snot": "will not",
        "ain\snot": "am not",
        "I\sam": "I am",
        "you\sare": "you are",
        "he\sis": "he is",
        "she\sis": "she is",
        "it\sis": "it is",
        "we\sare": "we are",
        "they\sare": "they are",
        "I\swould": "I would",
        "you\swould": "you would",
        "he\swould": "he would",
        "she\swould": "she would",
        "it\swould": "it would",
        "we\swould": "we would",
        "they\swould": "they would",
        "I\scould": "I could",
        "you\scould": "you could",
        "he\scould": "he could",
        "she\scould": "she could",
        "it\scould": "it could",
        "we\scould": "we could",
        "they\scould": "they could",
        "I\shave": "I have",
        "you\shave": "you have",
        "he\shas": "he has",
        "she\shas": "she has",
        "it\shas": "it has",
        "we\shave": "we have",
        "they\shave": "they have",
        "I\swill": "I will",
        "you\swill": "you will",
        "he\swill": "he will",
        "she\swill": "she will",
        "it\swill": "it will",
        "we\swill": "we will",
        "they\swill": "they will",
        "I\smust": "I must",
        "you\smust": "you must",
        "he\smust": "he must",
        "she\smust": "she must",
        "it\smust": "it must",
        "we\smust": "we must",
        "they\smust": "they must",
        "I\sshall": "I shall",
        "you\sshall": "you shall",
        "he\sshall": "he shall",
        "she\sshall": "she shall",
        "it\sshall": "it shall",
        "we\sshall": "we shall",
        "they\sshall": "they shall",
        "haven\s't": "have not",
        "hasn\s't": "has not",
        "hadn\s't": "had not",
        "don\s't": "do not",
        "doesn\s't": "does not",
        "didn\s't": "did not",
        "can\s't": "can not",
        "cannot": "can not",
        "couldn\s't": "could not",
        "shouldn\s't": "should not",
        "mightn\s't": "might not",
        "mustn\s't": "must not",
        "shan\s't": "shall not",
        "won\s't": "will not",
        "ain\s't": "am not",
        "aren\s't": "are not",
        "wasn\s't": "was not",
        "weren\s't": "were not",
        "I\sdidn't": "I did not",
        "you\sdidn't": "you did not",
        "he\sdidn't": "he did not",
        "she\sdidn't": "she did not",
        "it\sdidn't": "it did not",
        "we\sdidn't": "we did not",
        "they\sdidn't": "they did not",
        "I\scannot": "I can not",
        "you\scannot": "you can not",
        "he\scannot": "he can not",
        "she\scannot": "she can not",
        "it\scannot": "it can not",
        "we\scannot": "we can not",
        "they\scannot": "they can not",
        "I\swon't": "I will not",
        "you\swon't": "you will not",
        "he\swon't": "he will not",
        "she\swon't": "she will not",
        "it\swon't": "it will not",
        "we\swon't": "we will not",
        "they\swon't": "they will not",
        "I\shasn't": "I has not",
        "you\shasn't": "you has not",
        "he\shasn't": "he has not",
        "she\shasn't": "she has not",
        "it\shasn't": "it has not",
        "we\shasn't": "we has not",
        "they\shasn't": "they has not"
}

nots = {
    'not sad': 'Happy', 'not bad': 'Happy', 'not boring': 'Happy', 'not wrong': 'Happy', 'not bored': 'Happy',
        'not jealous': 'Happy', 'not happy': 'Sad', 'not well': 'Sad', 'not suitable': 'Angry',
        'not right': 'Angry', 'not good': 'Sad', 'not excited': 'Angry', 'not funny ': 'Sad', 'not kind': 'Sad',
        'not proud': 'Angry', 'not cool': 'Angry', 'not funny': 'Angry', 'not kind': 'Angry', 'not open': 'Angry',
        'not safe': 'Fear', 'not enough': 'Empty', 'not know': 'Sad', 'not knowing': 'Sad', 'not believe': 'Angry',
        'not believing': 'Angry', 'not understand': 'Sad', 'not understanding': 'Sad', 'no doubt': 'Happy',
        'not think': 'Sad', 'not thinking': 'Sad', 'not recognise': 'Sad', 'not recognising': 'Sad',
        'not forget': 'Angry', 'not forgetting': 'Angry', 'not remember': 'Sad', 'not remembering': 'Sad',
        'not imagine': 'Sad', 'not imagining': 'Sad', 'not mean': 'Sad', 'not meaning': 'Sad',
        'not agree': 'Angry', 'not agreeing': 'Sad', 'not disagree': 'Happy', 'not disagreeing': 'Happy',
        'not deny': 'Sad', 'not denying': 'Sad', 'not promise': 'Angry', 'not promising': 'Angry',
        'not satisfy': 'Sad', 'not satisfying': 'Sad', 'not realise': 'Sad', 'not realising': 'Sad',
        'not appear': 'Angry', 'not appearing': 'Angry', 'not please': 'Sad', 'not pleasing': 'Sad',
        'not impress': 'Sad', 'not impressing': 'Sad', 'not surprise': 'Sad', 'not surprising': 'Sad',
        'not concern': 'Sad', 'not concerning': 'Sad', 'not have': 'Sad', 'not having': 'Sad',
        'not own': 'Sad', 'not owning': 'Sad', 'not possess': 'Sad', 'not possessing': 'Sad',
        'not lack': 'Sad', 'not lacking': 'Sad', 'not consist': 'Sad', 'not consisting': 'Sad',
        'not involve': 'Sad', 'not involving': 'Sad', 'not include': 'Sad', 'not including': 'Sad',
        'not contain': 'Sad', 'not containing': 'Sad', 'not love': 'Sad', 'not like': 'Angry',
        'not hate': 'Happy', 'not hating': 'Happy', 'not adore': 'Sad', 'not adoring': 'Sad',
        'not prefer': 'Sad', 'not preferring': 'Sad', 'not care': 'Angry', 'not mind': 'Angry',
        'not minding': 'Sad', 'not want': 'Angry', 'not wanting': 'Sad', 'not need': 'Angry',
        'not needing': 'Angry', 'not desire': 'Sad', 'not desiring': 'Sad', 'not wish': 'Sad',
        'not wishing': 'Sad', 'not hope': 'Sad', 'not hoping': 'Sad', 'not appreciate': 'Sad',
        'not appreciating': 'Sad', 'not value': 'Sad', 'not valuing': 'Sad', 'not owe': 'Sad',
        'not owing': 'Sad', 'not seem': 'Sad', 'not seeming': 'Sad', 'not fit': 'Sad', 'not fitting': 'Sad',
        'not depend': 'Sad', 'not depending': 'Sad', 'not matter': 'Sad', 'not afford': 'Sad',
        'not affording': 'Sad', 'not aim': 'Sad', 'not aiming': 'Sad', 'not attempt': 'Angry',
        'not attempting': 'Angry', 'not ask': 'Angry', 'not asking': 'Angry', 'not arrange': 'Angry',
        'not arranging': 'Angry', 'not beg': 'Angry', 'not begging': 'Angry', 'not begin': 'Angry',
        'not beginning': 'Angry', 'not caring': 'Angry', 'not choose': 'Angry', 'not choosing': 'Angry',
        'not claim': 'Angry', 'not claiming': 'Angry', 'not consent': 'Angry', 'not consenting': 'Angry',
        'not continue': 'Angry', 'not continuing': 'Angry', 'not dare': 'Angry', 'not daring': 'Angry',
        'not decide': 'Sad', 'not deciding': 'Sad', 'not demand': 'Angry', 'not demanding': 'Angry',
        'not deserve': 'Angry', 'not deserving': 'Angry', 'not expect': 'Angry', 'not expecting': 'Angry',
        'not fail': 'Happy', 'not failing': 'Happy', 'not get': 'Sad', 'not getting': 'Sad',
        'not hesitate': 'Sad', 'not hesitating': 'Sad', 'not hurry': 'Happy', 'not hurrying': 'Happy',
        'not intend': 'Sad', 'not intending': 'Sad', 'not learn': 'Angry', 'not learning': 'Angry',
        'not liking': 'Angry', 'not loving': 'Sad', 'not manage': 'Angry', 'not managing': 'Angry',
        'not neglect': 'Sad', 'not neglecting': 'Sad', 'not offer': 'Angry', 'not offering': 'Angry',
        'not plan': 'Angry', 'not planing': 'Angry', 'not prepare': 'Angry', 'not preparing': 'Angry',
        'not pretend': 'Angry', 'not pretending': 'Angry', 'not proceed': 'Angry', 'not proceeding': 'Angry',
        'not propose': 'Angry', 'not proposing': 'Sad', 'not refuse': 'Sad', 'not refusing': 'Sad',
        'not start': 'Sad', 'not starting': 'Sad', 'not stop': 'Happy', 'not stopping': 'Happy',
        'not struggle': 'Angry', 'not struggling': 'Angry', 'not swear': 'Angry', 'not swearing': 'Angry',
        'not threaten': 'Happy', 'not threatening': 'Happy', 'not try': 'Angry', 'not trying': 'Angry',
        'not volunteer': 'Angry', 'not volunteering': 'Angry', 'not wait': 'Angry', 'not waiting': 'Angry',
        'not feel': 'Sad', 'not feeling': 'Sad', "not able": "Sad", "not do": "Sad",
        'not apologize': 'Sad', 'not apologizing': 'Sad', 'not forgive': 'Angry', 'not forgiving': 'Angry',
        'not trust': 'Angry', 'not trusting': 'Angry', 'not regret': 'Angry', 'not regretting': 'Angry',
        'not rejoice': 'Sad', 'not rejoicing': 'Sad', 'not admire': 'Sad', 'not admiring': 'Sad',
        'not compliment': 'Sad', 'not complimenting': 'Sad', 'not criticize': 'Happy', 'not criticizing': 'Happy',
        'not encourage': 'Angry', 'not encouraging': 'Angry', 'not insult': 'Sad', 'not insulting': 'Sad',
        'not praise': 'Angry', 'not praising': 'Angry', 'not support': 'Angry', 'not supporting': 'Angry',
        'not blame': 'Sad', 'not blaming': 'Sad', 'not defend': 'Sad', 'not defending': 'Sad',
        'not appreciate': 'Sad', 'not appreciating': 'Sad', 'not enjoy': 'Sad', 'not enjoying': 'Sad',
        'not like': 'Angry', 'not liking': 'Angry', 'not love': 'Sad', 'not loving': 'Sad',
        'not prefer': 'Sad', 'not preferring': 'Sad', 'not want': 'Angry', 'not wanting': 'Sad',
        'not believe': 'Angry', 'not believing': 'Angry', 'not doubt': 'Happy', 'not doubting': 'Happy',
        'not imagine': 'Sad', 'not imagining': 'Sad', 'not realize': 'Sad', 'not realizing': 'Sad',
        'not remember': 'Sad', 'not remembering': 'Sad', 'not recognize': 'Sad', 'not recognizing': 'Sad',
        'not consider': 'Sad', 'not considering': 'Sad', 'not think': 'Sad', 'not thinking': 'Sad',
        'not forget': 'Angry', 'not forgetting': 'Angry', 'not ignore': 'Angry', 'not ignoring': 'Angry',
        'not overlook': 'Angry', 'not overlooking': 'Angry', 'not understand': 'Sad', 'not understanding': 'Sad',
        'not hear': 'Angry', 'not hearing': 'Angry', 'not listen': 'Angry', 'not listening': 'Angry',
        'not look': 'Angry', 'not looking': 'Angry', 'not smell': 'Angry', 'not smelling': 'Angry',
        'not taste': 'Angry', 'not tasting': 'Angry', 'not touch': 'Angry', 'not touching': 'Angry',
        'not feel': 'Sad', 'not feeling': 'Sad', 'not sense': 'Sad', 'not sensing': 'Sad',
        'not suppose': 'Angry', 'not supposing': 'Angry', 'not expect': 'Angry', 'not expecting': 'Angry',
        'not wait': 'Angry', 'not waiting': 'Angry', 'not long': 'Angry', 'not longing': 'Angry',
        'not yearn': 'Angry', 'not yearning': 'Angry', 'not wish': 'Sad', 'not wishing': 'Sad',
        'not hope': 'Sad', 'not hoping': 'Sad', 'not desire': 'Sad', 'not desiring': 'Sad',
        'not miss': 'Angry', 'not missing': 'Angry', 'not need': 'Angry', 'not needing': 'Angry',
        'not want': 'Angry', 'not wanting': 'Sad', 'not require': 'Angry', 'not requiring': 'Angry',
        'not demand': 'Angry', 'not demanding': 'Angry', 'not insist': 'Angry', 'not insisting': 'Angry',
        'not force': 'Angry', 'not forcing': 'Angry', 'not push': 'Angry', 'not pushing': 'Angry',
        'not pull': 'Angry', 'not pulling': 'Angry', 'not drag': 'Angry', 'not dragging': 'Angry',
        'not carry': 'Angry', 'not carrying': 'Angry', 'not lift': 'Angry', 'not lifting': 'Angry',
        'not drop': 'Angry', 'not dropping': 'Angry', 'not throw': 'Angry', 'not throwing': 'Angry',
        'not catch': 'Angry', 'not catching': 'Angry', 'not capture': 'Angry', 'not capturing': 'Angry',
        'not grab': 'Angry', 'not grabbing': 'Angry', 'not touch': 'Angry', 'not touching': 'Angry',
        'not reach': 'Angry', 'not reaching': 'Angry', 'not approach': 'Angry', 'not approaching': 'Angry',
        'not avoid': 'Angry', 'not avoiding': 'Angry', 'not evade': 'Angry', 'not evading': 'Angry',
        'not elude': 'Angry', 'not eluding': 'Angry', 'not escape': 'Angry', 'not escaping': 'Angry',
        'not run': 'Angry', 'not running': 'Angry', 'not jog': 'Angry', 'not jogging': 'Angry',
        'not walk': 'Angry', 'not walking': 'Angry', 'not crawl': 'Angry', 'not crawling': 'Angry',
        'not sneak': 'Angry', 'not sneaking': 'Angry', 'not tiptoe': 'Angry', 'not tiptoeing': 'Angry',
        'not dance': 'Angry', 'not dancing': 'Angry', 'not stomp': 'Angry', 'not stomping': 'Angry',
        'not shake': 'Angry', 'not shaking': 'Angry', 'not tremble': 'Angry', 'not trembling': 'Angry',
        'not shiver': 'Angry', 'not shivering': 'Angry', 'not quiver': 'Angry', 'not quivering': 'Angry',
        'not vibrate': 'Angry', 'not vibrating': 'Angry', 'not pulsate': 'Angry', 'not pulsating': 'Angry',
        'not throb': 'Angry', 'not throbbing': 'Angry', 'not beat': 'Angry', 'not beating': 'Angry',
        'not palpitate': 'Angry', 'not palpitating': 'Angry', 'not pump': 'Angry', 'not pumping': 'Angry',
        'not glide': 'Angry', 'not gliding': 'Angry', 'not slide': 'Angry', 'not sliding': 'Angry',
        'not slip': 'Angry', 'not slipping': 'Angry', 'not skid': 'Angry'
}

shortcuts = {
    'u': 'you', 'y': 'why', 'r': 'are', 'doin': 'doing', 'hw': 'how', 'k': 'okay', 'm': 'am',
    'b4': 'before',
                   'idc': "i do not care", 'ty': 'thank you', 'wlcm': 'welcome', 'bc': 'because', '<3': 'love',
                   'xoxo': 'love',
                   'ttyl': 'talk to you later', 'gr8': 'great', 'bday': 'birthday', 'awsm': 'awesome', 'gud': 'good',
                   'h8': 'hate',
                   'lv': 'love', 'dm': 'direct message', 'rt': 'retweet', 'wtf': 'hate', 'idgaf': 'hate',
                   'irl': 'in real life', 'yolo': 'you only live once', "don't": "do not", 'g8': 'great',
                   "won't": "will not", 'tbh': 'to be honest', 'caj': 'casual', 'Ikr': 'I know, right?',
                   'omw': 'on my way',
                   'ofc': 'of course', 'Idc': "I don't care", 'Irl': 'In real life', 'tbf': 'To be fair',
                   'obvs': 'obviously', 'v': 'very', 'atm': 'at the moment',
                   'col': 'crying out loud', 'gbu': 'god bless you', 'gby': 'god bless you', 'gotcha': 'I got you',
                   'hehe': 'laughing', 'haha': 'laughing', 'hf': 'have fun',
                   'hry': 'hurry', 'hw': 'hardwork', 'idc': 'i don’t care', 'ikr': 'i know right', 'k': 'ok',
                   'lmao': 'laughing my ass off', 'lol': 'laughing out loud',
                   'n1': 'nice one', 'na': 'not available', 'qt': 'cutie', 'qtpi': 'cutie pie',
                   'rip': 'rest in peace',
                   'sry': 'sorry', 'tc': 'take care',
                   'thnks': 'thanks', 'thx': 'thanks', 'thnk': 'thanks', 'ttyl': 'talk to you later', 'txt': 'text',
                   'ugh': 'disgusted', 'w8': 'wait', "not sad": "happy"
}

### Importing the dataset

In [ ]:
dataset = pd.read_csv("/content/final_dataset.csv", header = None)
X = dataset[0].values
y = dataset[1].values

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y = le.fit_transform(y)
emotions = le.inverse_transform([0,1,2,3,4,5])
print(len(y))
print(emotions)

19999
['anger' 'fear' 'joy' 'love' 'sadness' 'surprise']


### Cleaning Texts

In [ ]:
def remove_contradictions(text):
    if "n't" in text:
        text = text.replace("n't", " not")
    for pattern, replacement in contractions.items():
        text = re.sub(pattern, replacement, text)
    return text

In [ ]:
def removing_not(text):
        f = re.findall("not\s\w+", text)
        for i in f:
            try:
                text = text.replace(i, nots[i])
            except:
                pass
        text = text.lower()
        return text

In [ ]:
def removing_shortcuts(text):
    full_words = []

    for token in text:
        if token in shortcuts.keys():
            token = shortcuts[token]
        full_words.append(token)
    text = " ".join(full_words)
    return text

In [ ]:
def removing_stopwords(text):
    return [word for word in text if not word in stop_words]

In [ ]:
corpus = []
for i in X:
  review = re.sub("[^a-zA-Z]"," ",i)
  review = review.lower()
  review = re.sub(r'http\S+|www.\S+', '', review)
  review = remove_contradictions(review)
  review = removing_not(review)
  review = review.split()
  review = removing_shortcuts(review)
  review = ' '.join([i for i in review.split() if not i.isdigit()])
  review = word_tokenize(review)
  review = removing_stopwords(review)
  lemma = WordNetLemmatizer()
  review = [lemma.lemmatize(word) for word in review]
  review = " ".join(review)
  corpus.append(review)

### Creating the Bag of Words

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=2000)
X = cv.fit_transform(corpus).toarray()
print(len(X),len(X[0]))

19999 2000


### Splitting the dataset into the Training set and Test set

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20)

## Classification Models

### MLP Model

In [ ]:
from sklearn.neural_network import MLPClassifier
MLP_classifier = MLPClassifier(max_iter = 1000)
MLP_classifier.fit(X_train, y_train)
y_pred = MLP_classifier.predict(X_test)
print(f"Accuracy = {accuracy_score(y_test, y_pred)*100:2.4f}%")
print("Classification Report:")
print(classification_report(y_test, y_pred, target_names=emotions))

Accuracy = 81.5711%
Classification Report:
              precision    recall  f1-score   support

       anger       0.75      0.74      0.74       683
        fear       0.77      0.77      0.77       520
       happy       0.85      0.86      0.86      1579
        love       0.74      0.73      0.74       303
     sadness       0.85      0.85      0.85      1403
    surprise       0.70      0.71      0.70       184

    accuracy                           0.82      4672
   macro avg       0.78      0.78      0.78      4672
weighted avg       0.82      0.82      0.82      4672



### GB Model

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
GB_classifier = GradientBoostingClassifier()
GB_classifier.fit(X_train, y_train)
y_pred = GB_classifier.predict(X_test)
print(f"Accuracy = {accuracy_score(y_test, y_pred)*100:2.4f}%")
print("Classification Report:")
print(classification_report(y_test, y_pred, target_names=emotions))

Accuracy = 78.5745%
Classification Report:
              precision    recall  f1-score   support

       anger       0.91      0.64      0.75       683
        fear       0.89      0.73      0.80       520
       happy       0.67      0.94      0.79      1579
        love       0.80      0.71      0.76       303
     sadness       0.92      0.71      0.80      1403
    surprise       0.77      0.79      0.78       184

    accuracy                           0.79      4672
   macro avg       0.83      0.76      0.78      4672
weighted avg       0.82      0.79      0.79      4672



### LightGBM Model

In [ ]:
from lightgbm import LGBMClassifier
LGBM_classifier = LGBMClassifier()
LGBM_classifier.fit(X_train, y_train)
y_pred = LGBM_classifier.predict(X_test)
print(f"Accuracy = {accuracy_score(y_test, y_pred)*100:2.4f}%")
print("Classification Report:")
print(classification_report(y_test, y_pred, target_names=emotions))

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.138465 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3641
[LightGBM] [Info] Number of data points in the train set: 18688, number of used features: 1342
[LightGBM] [Info] Start training from score -1.892919
[LightGBM] [Info] Start training from score -2.173169
[LightGBM] [Info] Start training from score -1.088603
[LightGBM] [Info] Start training from score -2.635212
[LightGBM] [Info] Start training from score -1.237417
[LightGBM] [Info] Start training from score -3.296051
Accuracy = 84.0539%
Classification Report:
              precision    recall  f1-score   support

       anger       0.79      0.77      0.78       683
        fear       0.85      0.81      0.83       520
       happy       0.82      0.89      0.85      1579
        love       0.77      0.82      0.79       303
     sa

### CatBoost Model

In [ ]:
from catboost import CatBoostClassifier
CB_classifier = CatBoostClassifier(logging_level='Silent')
CB_classifier.fit(X_train, y_train)
y_pred = CB_classifier.predict(X_test)
print(f"Accuracy = {accuracy_score(y_test, y_pred)*100:2.4f}%")
print("Classification Report:")
print(classification_report(y_test, y_pred, target_names=emotions))

Accuracy = 84.8673%
Classification Report:
              precision    recall  f1-score   support

       anger       0.88      0.75      0.81       683
        fear       0.85      0.81      0.83       520
       happy       0.80      0.92      0.86      1579
        love       0.82      0.70      0.76       303
     sadness       0.91      0.87      0.89      1403
    surprise       0.83      0.73      0.78       184

    accuracy                           0.85      4672
   macro avg       0.85      0.80      0.82      4672
weighted avg       0.85      0.85      0.85      4672



### XGBoost Model

In [ ]:
from xgboost import XGBClassifier
XGBC_classifier = XGBClassifier()
XGBC_classifier.fit(X_train, y_train)
y_pred = XGBC_classifier.predict(X_test)
print(f"Accuracy = {accuracy_score(y_test, y_pred)*100:2.4f}%")
print("Classification Report:")
print(classification_report(y_test, y_pred, target_names=emotions))

Accuracy = 84.8887%
Classification Report:
              precision    recall  f1-score   support

       anger       0.87      0.77      0.82       683
        fear       0.87      0.82      0.84       520
       happy       0.81      0.90      0.85      1579
        love       0.76      0.83      0.80       303
     sadness       0.92      0.85      0.88      1403
    surprise       0.79      0.77      0.78       184

    accuracy                           0.85      4672
   macro avg       0.84      0.82      0.83      4672
weighted avg       0.85      0.85      0.85      4672



### Extra Trees Model

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier
ET_classifier = ExtraTreesClassifier()
ET_classifier.fit(X_train, y_train)
y_pred = ET_classifier.predict(X_test)
print(f"Accuracy = {accuracy_score(y_test, y_pred)*100:2.4f}%")
print("Classification Report:")
print(classification_report(y_test, y_pred, target_names=emotions))

Accuracy = 84.9101%
Classification Report:
              precision    recall  f1-score   support

       anger       0.79      0.80      0.79       683
        fear       0.82      0.82      0.82       520
       happy       0.89      0.86      0.88      1579
        love       0.77      0.78      0.77       303
     sadness       0.88      0.89      0.88      1403
    surprise       0.75      0.80      0.77       184

    accuracy                           0.85      4672
   macro avg       0.81      0.83      0.82      4672
weighted avg       0.85      0.85      0.85      4672



### Random Forest Classification


In [ ]:
from sklearn.ensemble import RandomForestClassifier
random_forest_classifier = RandomForestClassifier()
random_forest_classifier.fit(X_train, y_train)
y_pred = random_forest_classifier.predict(X_test)
print(f"Accuracy = {accuracy_score(y_test, y_pred)*100:2.4f}%")
print("Classification Report:")
print(classification_report(y_test, y_pred, target_names=emotions))

Accuracy = 83.7115%
Classification Report:
              precision    recall  f1-score   support

       anger       0.76      0.82      0.79       683
        fear       0.82      0.81      0.81       520
       happy       0.89      0.84      0.86      1579
        love       0.74      0.78      0.76       303
     sadness       0.87      0.87      0.87      1403
    surprise       0.73      0.79      0.76       184

    accuracy                           0.84      4672
   macro avg       0.80      0.82      0.81      4672
weighted avg       0.84      0.84      0.84      4672



### Decision Tree Model

In [ ]:
from sklearn.tree import DecisionTreeClassifier
decision_classifier = DecisionTreeClassifier()
decision_classifier.fit(X_train, y_train)
y_pred = decision_classifier.predict(X_test)
print(f"Accuracy = {accuracy_score(y_test, y_pred)*100:2.4f}%")
print("Classification Report:")
print(classification_report(y_test, y_pred, target_names=emotions))

Accuracy = 79.0882%
Classification Report:
              precision    recall  f1-score   support

       anger       0.69      0.82      0.75       683
        fear       0.79      0.79      0.79       520
       happy       0.86      0.74      0.79      1579
        love       0.69      0.77      0.73       303
     sadness       0.82      0.84      0.83      1403
    surprise       0.69      0.79      0.74       184

    accuracy                           0.79      4672
   macro avg       0.76      0.79      0.77      4672
weighted avg       0.80      0.79      0.79      4672



### Logistic Regression model

In [ ]:
from sklearn.linear_model import LogisticRegression
logistic_classifier = LogisticRegression(max_iter = 1000)
logistic_classifier.fit(X_train, y_train)
y_pred = logistic_classifier.predict(X_test)
print(f"Accuracy = {accuracy_score(y_test, y_pred)*100:2.4f}%")
print("Classification Report:")
print(classification_report(y_test, y_pred, target_names=emotions))

Accuracy = 84.6104%
Classification Report:
              precision    recall  f1-score   support

       anger       0.81      0.78      0.79       683
        fear       0.86      0.79      0.83       520
       happy       0.85      0.89      0.87      1579
        love       0.79      0.72      0.76       303
     sadness       0.86      0.89      0.88      1403
    surprise       0.86      0.70      0.77       184

    accuracy                           0.85      4672
   macro avg       0.84      0.80      0.82      4672
weighted avg       0.85      0.85      0.84      4672



### Kernal SVM

In [ ]:
from sklearn.svm import SVC
kernal_classifier = SVC(kernel = 'rbf')
kernal_classifier.fit(X_train, y_train)
y_pred = kernal_classifier.predict(X_test)
print(f"Accuracy = {accuracy_score(y_test, y_pred)*100:2.4f}%")
print("Classification Report:")
print(classification_report(y_test, y_pred, target_names=emotions))

Accuracy = 81.7637%
Classification Report:
              precision    recall  f1-score   support

       anger       0.84      0.68      0.75       683
        fear       0.83      0.75      0.79       520
       happy       0.79      0.92      0.85      1579
        love       0.86      0.56      0.68       303
     sadness       0.83      0.89      0.86      1403
    surprise       0.90      0.53      0.67       184

    accuracy                           0.82      4672
   macro avg       0.84      0.72      0.77      4672
weighted avg       0.82      0.82      0.81      4672



### Support Vector Machine

In [ ]:
from sklearn.svm import SVC
svm_classifier = SVC(kernel = 'linear')
svm_classifier.fit(X_train, y_train)
y_pred = svm_classifier.predict(X_test)
print(f"Accuracy = {accuracy_score(y_test, y_pred)*100:2.4f}%")
print("Classification Report:")
print(classification_report(y_test, y_pred, target_names=emotions))

Accuracy = 83.3904%
Classification Report:
              precision    recall  f1-score   support

       anger       0.77      0.78      0.78       683
        fear       0.84      0.81      0.82       520
       happy       0.84      0.87      0.86      1579
        love       0.76      0.75      0.76       303
     sadness       0.87      0.86      0.87      1403
    surprise       0.80      0.72      0.76       184

    accuracy                           0.83      4672
   macro avg       0.81      0.80      0.81      4672
weighted avg       0.83      0.83      0.83      4672



### K-Nearest Neighbors (K-NN)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
K_nearest_classifier = KNeighborsClassifier()
K_nearest_classifier.fit(X_train, y_train)
y_pred = K_nearest_classifier.predict(X_test)
print(f"Accuracy = {accuracy_score(y_test, y_pred)*100:2.4f}%")
print("Classification Report:")
print(classification_report(y_test, y_pred, target_names=emotions))

Accuracy = 53.8527%
Classification Report:
              precision    recall  f1-score   support

       anger       0.41      0.60      0.49       683
        fear       0.50      0.42      0.46       520
       happy       0.63      0.59      0.61      1579
        love       0.51      0.28      0.37       303
     sadness       0.55      0.59      0.57      1403
    surprise       0.49      0.21      0.30       184

    accuracy                           0.54      4672
   macro avg       0.52      0.45      0.46      4672
weighted avg       0.55      0.54      0.53      4672



### Naive Bayes model

In [ ]:
from sklearn.naive_bayes import GaussianNB
naive_classifier = GaussianNB()
naive_classifier.fit(X_train,y_train)
y_pred = naive_classifier.predict(X_test)
print(f"Accuracy = {accuracy_score(y_test, y_pred)*100:2.4f}%")
print("Classification Report:")
print(classification_report(y_test, y_pred, target_names=emotions))

Accuracy = 37.7140%
Classification Report:
              precision    recall  f1-score   support

       anger       0.53      0.42      0.47       683
        fear       0.30      0.52      0.38       520
       happy       0.84      0.30      0.44      1579
        love       0.18      0.70      0.28       303
     sadness       0.82      0.29      0.43      1403
    surprise       0.11      0.60      0.19       184

    accuracy                           0.38      4672
   macro avg       0.46      0.47      0.37      4672
weighted avg       0.66      0.38      0.42      4672



## Voting Classifier Model

In [ ]:
from sklearn.ensemble import VotingClassifier

from sklearn.neural_network import MLPClassifier
MLP_classifier = MLPClassifier(max_iter = 1000)

from sklearn.ensemble import GradientBoostingClassifier
GB_classifier = GradientBoostingClassifier()

from lightgbm import LGBMClassifier
LGBM_classifier = LGBMClassifier()

from xgboost import XGBClassifier
XGBC_classifier = XGBClassifier()

from sklearn.ensemble import ExtraTreesClassifier
ET_classifier = ExtraTreesClassifier()

from sklearn.ensemble import RandomForestClassifier
random_forest_classifier = RandomForestClassifier()

from sklearn.tree import DecisionTreeClassifier
decision_classifier = DecisionTreeClassifier()

from sklearn.linear_model import LogisticRegression
logistic_classifier = LogisticRegression(max_iter = 1000)

from sklearn.svm import SVC
kernal_classifier = SVC(kernel = 'rbf')

from sklearn.svm import SVC
svm_classifier = SVC(kernel = 'linear')


ensemble = VotingClassifier(
    estimators=[
        ('MLPC', MLP_classifier),
        ('GBC', GB_classifier),
        ('LGBMC', LGBM_classifier),
        ('XGBC', XGBC_classifier),
        ('ETC', ET_classifier),
        ('RFC', random_forest_classifier),
        ('DTC', decision_classifier),
        ('LC', logistic_classifier),
        ('KSVMC', kernal_classifier),
        ('SVMC', svm_classifier)
        ],
    voting='hard')
ensemble.fit(X_train, y_train)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.191238 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3167
[LightGBM] [Info] Number of data points in the train set: 15999, number of used features: 1188
[LightGBM] [Info] Start training from score -1.998721
[LightGBM] [Info] Start training from score -2.129620
[LightGBM] [Info] Start training from score -1.086868
[LightGBM] [Info] Start training from score -2.497170
[LightGBM] [Info] Start training from score -1.237812
[LightGBM] [Info] Start training from score -3.324174


VotingClassifier(estimators=[('MLPC', MLPClassifier(max_iter=1000)),
                             ('GBC', GradientBoostingClassifier()),
                             ('LGBMC', LGBMClassifier()),
                             ('XGBC',
                              XGBClassifier(base_score=None, booster=None,
                                            callbacks=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=None, device=None,
                                            early_stopping_rounds=None,
                                            enable_categorical=False,
                                            eval_metric=None,
                                            featu...
                                            max_leaves=None,
                                            min_child_weight=None, missing=nan,
                                            monotone_constraints=None,
                                            multi_strategy=None,
                                            n_estimators=None, n_jobs=None,
                                            num_parallel_tree=None,
                                            random_state=None, ...)),
                             ('ETC', ExtraTreesClassifier()),
                             ('RFC', RandomForestClassifier()),
                             ('DTC', DecisionTreeClassifier()),
                             ('LC', LogisticRegression(max_iter=1000)),
                             ('KSVMC', SVC()), ('SVMC', SVC(kernel='linear'))])

In [ ]:
y_pred = ensemble.predict(X_test)
print(f"Accuracy = {accuracy_score(y_test, y_pred)*100:2.4f}%")
print("Classification Report:")
print(classification_report(y_test, y_pred, target_names=emotions))

Accuracy = 89.1250%
Classification Report:
              precision    recall  f1-score   support

       anger       0.88      0.90      0.89       541
        fear       0.85      0.86      0.85       471
         joy       0.90      0.93      0.91      1364
        love       0.82      0.76      0.79       324
     sadness       0.94      0.92      0.93      1157
    surprise       0.77      0.69      0.73       143

    accuracy                           0.89      4000
   macro avg       0.86      0.84      0.85      4000
weighted avg       0.89      0.89      0.89      4000



In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

In [ ]:
classifiers = [['Neural Network :', MLPClassifier(max_iter = 1000)],
               ['GradientBoostingClassifier: ', GradientBoostingClassifier()],
               ['LGBM_classifier: ', LGBMClassifier()],
               ['CatBoost :', CatBoostClassifier(logging_level='Silent')],
               ['XGB :', XGBClassifier()],
               ['ExtraTreesClassifier :', ExtraTreesClassifier()],
               ['RandomForest :',RandomForestClassifier()],
               ['DecisionTree :',DecisionTreeClassifier()],
               ['LogisticRegression :', LogisticRegression(max_iter = 1000)],
               ['Kernel SVM :', SVC(kernel = 'rbf')],
               ['Linear SVM :', SVC(kernel = "linear")],
               ['SVM :', SVC()],
               ['KNeighbours :', KNeighborsClassifier()],
               ['Naive Bayes :', GaussianNB()],
               ['AdaBoostClassifier :', AdaBoostClassifier()],
               ]

for name,classifier in classifiers:
    classifier = classifier
    classifier.fit(X_train, y_train)
    y_pred = classifier.predict(X_test)
    print(name, accuracy_score(y_test, y_pred))

Neural Network : 0.84775
GradientBoostingClassifier:  0.847
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.119011 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3167
[LightGBM] [Info] Number of data points in the train set: 15999, number of used features: 1188
[LightGBM] [Info] Start training from score -1.998721
[LightGBM] [Info] Start training from score -2.129620
[LightGBM] [Info] Start training from score -1.086868
[LightGBM] [Info] Start training from score -2.497170
[LightGBM] [Info] Start training from score -1.237812
[LightGBM] [Info] Start training from score -3.324174
LGBM_classifier:  0.87775
CatBoost : 0.88925
XGB : 0.89275
ExtraTreesClassifier : 0.88475
RandomForest : 0.8765
DecisionTree : 0.83725
LogisticRegression : 0.884
Kernel SVM : 0.83575
Linear SVM : 0.868
SVM : 0.83575
KNeighbours : 0.573
Naive Bayes : 0.395
AdaBo

## Predict Single Result

In [ ]:
def predict_emotion(text, classifier):
  text = text.replace("\n"," ")
  review = re.sub("[^a-zA-Z]"," ",text)
  review = review.lower()
  review = re.sub(r'http\S+|www.\S+', '', review)
  review = remove_contradictions(review)
  review = removing_not(review)
  review = review.split()
  review = removing_shortcuts(review)
  review = ' '.join([i for i in review.split() if not i.isdigit()])
  review = word_tokenize(review)
  review = removing_stopwords(review)
  lemma = WordNetLemmatizer()
  review = [lemma.lemmatize(word) for word in review]
  review = " ".join(review)
  test = cv.transform([review]).toarray()
  result = classifier.predict(test)
  result = le.inverse_transform(result)
  return result

In [ ]:
text = '''I am not happy to see you here .'''

print(predict_emotion(text, ensemble))
print(predict_emotion(text, GB_classifier))
print(predict_emotion(text, LGBM_classifier))
print(predict_emotion(text, CB_classifier))
print(predict_emotion(text, XGBC_classifier))
print(predict_emotion(text, ET_classifier))
print(predict_emotion(text, random_forest_classifier))
print(predict_emotion(text, decision_classifier))
print(predict_emotion(text, K_nearest_classifier))
print(predict_emotion(text, kernal_classifier))
print(predict_emotion(text, svm_classifier))
print(predict_emotion(text, naive_classifier))
print(predict_emotion(text, logistic_classifier))

['joy']


NotFittedError: ignored